In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score

from tqdm import tqdm

import warnings; warnings.filterwarnings('ignore')

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score


import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import random
import os

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
from pathlib import Path
DATA_PATH = Path("C:/ML_Projects/kaggle-workflow/data/")
OUTPUT_PATH = Path("C:/ML_Projects/kaggle-workflow/output/")

# General settings
SEED = 42
N_FOLDS = 5

# Metric choice placeholder
#Adjust depending on the competition
METRIC = "auc"

In [4]:
#Load Data
df = pd.read_csv(DATA_PATH / "Diabetes Prediction Challenge" / "train.csv")

In [5]:
df.columns

Index(['id', 'age', 'alcohol_consumption_per_week',
       'physical_activity_minutes_per_week', 'diet_score',
       'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi',
       'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate',
       'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol',
       'triglycerides', 'gender', 'ethnicity', 'education_level',
       'income_level', 'smoking_status', 'employment_status',
       'family_history_diabetes', 'hypertension_history',
       'cardiovascular_history', 'diagnosed_diabetes'],
      dtype='object')

In [6]:
df = df.rename(columns = ({"alcohol_consumption_per_week":"alc",'physical_activity_minutes_per_week':"activity", 'diet_score':"diet",
                          'sleep_hours_per_day': "sleep", 'screen_time_hours_per_day':"screen", 'education_level':"edu",
                           'income_level':"inc", 'smoking_status': "smoke", 'employment_status':"empl", 'family_history_diabetes': "fam_his",
                          'hypertension_history': "hyp_his", 'cardiovascular_history': "card_his", 'diagnosed_diabetes': "label"} ))

In [7]:
# === Basic EDA ===

# Shape
print("Shape:", df.shape)

# Preview
display(df.head())

# Info
print("\nInfo:")
df.info()

# Target distribution 
print("\nTarget distribution:")
print(df["label"].value_counts(normalize=True))

# Numerical stats
print("\nNumerical summary:")
display(df.describe())

# Categorical stats (top categories)
print("\nCategorical summary:")
for col in df.select_dtypes(include=["object", "category"]).columns:
    print(f"{col}: {df[col].nunique()} unique")


Shape: (700000, 26)


,id,age,alc,activity,diet,sleep,screen,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,edu,inc,smoke,empl,fam_his,hyp_his,card_his,label
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,...,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,...,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,...,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,...,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,...,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 26 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  700000 non-null  int64  
 1   age                 700000 non-null  int64  
 2   alc                 700000 non-null  int64  
 3   activity            700000 non-null  int64  
 4   diet                700000 non-null  float64
 5   sleep               700000 non-null  float64
 6   screen              700000 non-null  float64
 7   bmi                 700000 non-null  float64
 8   waist_to_hip_ratio  700000 non-null  float64
 9   systolic_bp         700000 non-null  int64  
 10  diastolic_bp        700000 non-null  int64  
 11  heart_rate          700000 non-null  int64  
 12  cholesterol_total   700000 non-null  int64  
 13  hdl_cholesterol     700000 non-null  int64  
 14  ldl_cholesterol     700000 non-null  int64  
 15  triglycerides       700000 

,id,age,alc,activity,diet,sleep,screen,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,fam_his,hyp_his,card_his,label
count,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000
mean,349999.500000,50.359734,2.072411,80.230803,5.963695,7.002200,6.012733,25.874684,0.858766,116.294193,75.440924,70.167749,186.818801,53.823214,102.905854,123.081850,0.149401,0.181990,0.030324,0.623296
std,202072.738554,11.655520,1.048189,51.195071,1.463336,0.901907,2.022707,2.860705,0.037980,11.010390,6.825775,6.938722,16.730832,8.266545,19.022416,24.739397,0.356484,0.385837,0.171478,0.484560
min,0.000000,19.000000,1.000000,1.000000,0.100000,3.100000,0.600000,15.100000,0.680000,91.000000,51.000000,42.000000,117.000000,21.000000,51.000000,31.000000,0.000000,0.000000,0.000000,0.000000
25%,174999.750000,42.000000,1.000000,49.000000,5.000000,6.400000,4.600000,23.900000,0.830000,108.000000,71.000000,65.000000,175.000000,48.000000,89.000000,106.000000,0.000000,0.000000,0.000000,0.000000
50%,349999.500000,50.000000,2.000000,71.000000,6.000000,7.000000,6.000000,25.900000,0.860000,116.000000,75.000000,70.000000,187.000000,54.000000,103.000000,123.000000,0.000000,0.000000,0.000000,1.000000
75%,524999.250000,58.000000,3.000000,96.000000,7.000000,7.600000,7.400000,27.800000,0.880000,124.000000,80.000000,75.000000,199.000000,59.000000,116.000000,139.000000,0.000000,0.000000,0.000000,1.000000
max,699999.000000,89.000000,9.000000,747.000000,9.900000,9.900000,16.500000,38.400000,1.050000,163.000000,104.000000,101.000000,289.000000,90.000000,205.000000,290.000000,1.000000,1.000000,1.000000,1.000000



Categorical summary:
gender: 3 unique
ethnicity: 5 unique
edu: 4 unique
inc: 5 unique
smoke: 3 unique
empl: 4 unique


In [8]:
df.columns

Index(['id', 'age', 'alc', 'activity', 'diet', 'sleep', 'screen', 'bmi',
       'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate',
       'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol',
       'triglycerides', 'gender', 'ethnicity', 'edu', 'inc', 'smoke', 'empl',
       'fam_his', 'hyp_his', 'card_his', 'label'],
      dtype='object')

In [9]:
target_col = ['label']
use_cols = ['age', 'activity', 'diet','screen', 'bmi', 'waist_to_hip_ratio', 'systolic_bp', 'heart_rate',
       'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol',
       'triglycerides', 'gender', 'ethnicity', 'edu', 'inc', 'smoke', 'empl',
       'fam_his']
X = df[use_cols].copy()
y=df[target_col].astype(int).copy()

In [10]:
cat_features =['gender', 'ethnicity', 'edu', 'inc', 'smoke', 'empl']

num_features = X.select_dtypes(include=["int64","float64"]).columns

In [11]:
X = df[num_features]

In [12]:
final_num = X.columns.tolist()
final_num

['age',
 'activity',
 'diet',
 'screen',
 'bmi',
 'waist_to_hip_ratio',
 'systolic_bp',
 'heart_rate',
 'cholesterol_total',
 'hdl_cholesterol',
 'ldl_cholesterol',
 'triglycerides',
 'fam_his']

In [13]:
X["age*activity"] = X["age"] * X["activity"]
X.head()

,age,activity,diet,screen,bmi,waist_to_hip_ratio,systolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,fam_his,age*activity
0,31,45,7.7,6.1,33.4,0.93,112,62,199,58,114,102,0,1395
1,50,73,5.7,5.8,23.8,0.83,120,71,199,50,121,124,0,3650
2,32,158,8.5,9.1,24.1,0.83,95,73,188,59,114,108,0,5056
3,54,77,4.6,9.2,26.6,0.83,121,74,182,54,85,123,0,4158
4,54,55,5.7,5.1,28.8,0.90,108,85,206,49,131,124,0,2970


In [14]:
final_cols = X.columns.tolist()
final_cols

['age',
 'activity',
 'diet',
 'screen',
 'bmi',
 'waist_to_hip_ratio',
 'systolic_bp',
 'heart_rate',
 'cholesterol_total',
 'hdl_cholesterol',
 'ldl_cholesterol',
 'triglycerides',
 'fam_his',
 'age*activity']

In [15]:
model = LGBMClassifier(
        n_estimators=500,
        learning_rate=0.03,
        num_leaves=31,
        min_data_in_leaf = 60,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity = -1
    )

In [16]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(X))

for train_idx, val_idx in kf.split(X, y):
    X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

    model = LGBMClassifier(
        n_estimators=500,
        learning_rate=0.03,
        num_leaves=31,
        min_data_in_leaf = 60,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity = -1
    )

    model.fit(X_tr, y_tr)
    oof[val_idx] = model.predict_proba(X_va)[:,1]

In [17]:
oof_lgbm = oof

In [24]:
ENS_PATH = Path("C:\ML_Projects\kaggle-workflow\competitions\Diabetes Prediction Challenge\Ens")
oof_models = pd.read_csv(ENS_PATH / "oof_predictions.csv")
oof_models.head()

,id,target,oof_pred_MLP
0,0,1.0,0.608593
1,1,1.0,0.644023
2,2,0.0,0.292208
3,3,1.0,0.666945
4,4,1.0,0.748159


In [25]:
oof_models["oof_pred_LGBM"] = oof_lgbm
oof_models.head()

,id,target,oof_pred_MLP,oof_pred_LGBM
0,0,1.0,0.608593,0.498156
1,1,1.0,0.644023,0.592414
2,2,0.0,0.292208,0.248972
3,3,1.0,0.666945,0.533050
4,4,1.0,0.748159,0.776689


In [26]:
oof_models.to_csv("oof_predictions.csv", index=False)